In [9]:
from dataprep import dataprep
from modeltrain import modeltrain
from hmm_test import hmm_test

import numpy as np

In [3]:
train_data, test_data, labels = dataprep("database_inc_sampchar", 5)

hmm_learn, train_acc = modeltrain(train_data, labels, 20)

Database read is  database_inc_sampchar
Labels used are  ['A', 'C', 'K', 'P', 'X', 'T', '+', 'N', 'V', '4']

 ------------ CHARACTER  A ------------
Estimated q:
[1. 0.]

Estimated A:
[[9.54268293e-001 4.57317073e-002]
 [2.22878796e-112 1.00000000e+000]]

Estimated means:
[[ 9.88769976e+00  1.30798728e-02]
 [ 3.73023796e+01 -9.37072087e+00]]

Estimated covariances:
[[[2.61255325e+00 1.94903689e+01]
  [1.94903689e+01 4.02569848e+03]]

 [[2.14526648e+01 1.08125268e+01]
  [1.08125268e+01 2.36881247e+02]]]
Avg probability over test samples is 0.017219018282242204

 ------------ CHARACTER  C ------------
Estimated q:
[6.62724005e-02 8.98473522e-04 9.32829126e-01]

Estimated A:
[[2.47463913e-01 2.24611127e-01 5.27924961e-01]
 [2.04612237e-49 8.40078413e-01 1.59921587e-01]
 [1.76327292e-01 7.35095721e-02 7.50163135e-01]]

Estimated means:
[[  9.68231951  -2.16599762]
 [  9.72874505  60.43510742]
 [  9.93011164 -24.33550863]]

Estimated covariances:
[[[2.00577132e-01 1.34768254e+00]
  [1.34768

In [4]:
acc, res_labl_list = hmm_test(hmm_learn, test_data, labels )

Classification accuracy of test samples of character A is: 100.0%
Classification accuracy of test samples of character C is: 0.0%
Classification accuracy of test samples of character K is: 0.0%
Classification accuracy of test samples of character P is: 0.0%
Classification accuracy of test samples of character X is: 80.0%
Classification accuracy of test samples of character T is: 0.0%
Classification accuracy of test samples of character + is: 40.0%
Classification accuracy of test samples of character N is: 0.0%
Classification accuracy of test samples of character V is: 0.0%
Classification accuracy of test samples of character 4 is: 0.0%


In [13]:
print("---------------- RESULTS: ---------------- ")
for k in range(len(labels)):
    print("\n Character: ", labels[k])
    print("Test accuracy: ",acc[k], "and resulting labels: ",res_labl_list[k])
    print("Training accuracy: (log)", train_acc[k])
    

---------------- RESULTS: ---------------- 

 Character:  A
Test accuracy:  1.0 and resulting labels:  ['A', 'A', 'A', 'A', 'A']
Training accuracy: (log) -4.061740791930455

 Character:  C
Test accuracy:  0.0 and resulting labels:  ['A', 'A', 'A', 'A', 'A']
Training accuracy: (log) -12.413007458587364

 Character:  K
Test accuracy:  0.0 and resulting labels:  ['A', 'A', 'A', 'A', 'A']
Training accuracy: (log) -6.355754420964

 Character:  P
Test accuracy:  0.0 and resulting labels:  ['A', 'A', 'A', 'A', 'A']
Training accuracy: (log) -6.945116634799745

 Character:  X
Test accuracy:  0.8 and resulting labels:  ['X', 'X', 'X', '+', 'X']
Training accuracy: (log) -3.369676426343301

 Character:  T
Test accuracy:  0.0 and resulting labels:  ['X', '+', '+', '+', 'X']
Training accuracy: (log) -11.38230089872172

 Character:  +
Test accuracy:  0.4 and resulting labels:  ['X', '+', 'X', 'X', '+']
Training accuracy: (log) -3.312126645211245

 Character:  N
Test accuracy:  0.0 and resulting label